In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')


plt.style.use('default')
sns.set_palette("husl")

print("="*50)
print("PREDICCIÓN DE PRECIOS DE CASAS EN CALIFORNIA - REGRESIÓN RIDGE")
print("="*50)

# 1. CARGA Y EXPLORACIÓN DEL DATASET
print("\n1. CARGANDO DATASET...")
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['PRICE'] = california.target

print(f"\nDimensiones del dataset: {df.shape}")
print(f"\nDescripción del dataset:")
print(california.DESCR[:500] + "...")
print(f"\nPrimeras filas:")
print(df.head(10))
print(f"\nInformación del dataset:")
print(df.info())
print(f"\nEstadísticas descriptivas:")
print(df.describe().round(2))

# Variables del dataset
print(f"\n{'='*70}")
print("VARIABLES DEL DATASET:")
print(f"{'='*70}")
for i, feature in enumerate(california.feature_names):
    print(f"{i+1}. {feature}")
print(f"\nVariable objetivo: PRICE (precio medio de las casas en $100,000)")

# 2. ANÁLISIS EXPLORATORIO
print(f"\n{'='*70}")
print("2. ANÁLISIS EXPLORATORIO DE DATOS")
print(f"{'='*70}")

# Valores nulos
print(f"\nValores nulos:")
print(df.isnull().sum())

# Matriz de correlación
print("\nCalculando correlaciones con la variable objetivo...")
correlations = df.corr()['PRICE'].sort_values(ascending=False)
print("\nCorrelaciones con PRICE:")
print(correlations)

# 3. PREPARACIÓN DE DATOS
print(f"\n{'='*50}")
print("3. PREPARACIÓN DE DATOS")
print(f"{'='*50}")

# Separar características (X) y variable objetivo (y)
X = df.drop('PRICE', axis=1)
y = df['PRICE']

print(f"\nForma de X (características): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")

# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nConjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")

# Normalización de características (importante para Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Datos normalizados con StandardScaler")
print(f"  Media de características en train: {X_train_scaled.mean():.6f}")
print(f"  Std de características en train: {X_train_scaled.std():.6f}")

# 4. ENTRENAMIENTO DEL MODELO RIDGE
print(f"\n{'='*50}")
print("4. ENTRENAMIENTO DEL MODELO RIDGE")
print(f"{'='*50}")

# Búsqueda del mejor hiperparámetro alpha usando GridSearchCV
print("\nBuscando el mejor hiperparámetro alpha...")
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = {'alpha': alphas}

ridge_cv = GridSearchCV(
    Ridge(random_state=42),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

ridge_cv.fit(X_train_scaled, y_train)

print(f"\n✓ Mejor alpha encontrado: {ridge_cv.best_params_['alpha']}")
print(f"  Mejor MSE (validación cruzada): {-ridge_cv.best_score_:.4f}")

# Entrenar modelo final con mejor alpha
best_alpha = ridge_cv.best_params_['alpha']
ridge_model = Ridge(alpha=best_alpha, random_state=42)
ridge_model.fit(X_train_scaled, y_train)

print(f"\n✓ Modelo Ridge entrenado exitosamente")

# Comparación con Regresión Lineal simple
print("\nEntrenando modelo de Regresión Lineal para comparación...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
print("✓ Modelo de Regresión Lineal entrenado")

# 5. EVALUACIÓN DEL MODELO
print(f"\n{'='*70}")
print("5. EVALUACIÓN DEL MODELO")
print(f"{'='*70}")

# Predicciones
y_train_pred_ridge = ridge_model.predict(X_train_scaled)
y_test_pred_ridge = ridge_model.predict(X_test_scaled)

y_train_pred_lr = lr_model.predict(X_train_scaled)
y_test_pred_lr = lr_model.predict(X_test_scaled)

# Cálculo de métricas para Ridge
print("\nMÉTRICAS DEL MODELO RIDGE:")
print("-" * 50)

# MSE (Error Cuadrático Medio)
mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge)
print(f"MSE (Mean Squared Error):")
print(f"  - Entrenamiento: {mse_train_ridge:.4f}")
print(f"  - Prueba: {mse_test_ridge:.4f}")

# RMSE (Raíz del Error Cuadrático Medio)
rmse_train_ridge = np.sqrt(mse_train_ridge)
rmse_test_ridge = np.sqrt(mse_test_ridge)
print(f"\nRMSE (Root Mean Squared Error):")
print(f"  - Entrenamiento: {rmse_train_ridge:.4f} ($100k)")
print(f"  - Prueba: {rmse_test_ridge:.4f} ($100k)")

# R² (Coeficiente de Determinación)
r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)
print(f"\nR² Score (Coeficiente de Determinación):")
print(f"  - Entrenamiento: {r2_train_ridge:.4f}")
print(f"  - Prueba: {r2_test_ridge:.4f}")
print(f"  Interpretación: El modelo explica {r2_test_ridge*100:.2f}% de la varianza")

# MAE (Error Absoluto Medio)
mae_train_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
print(f"\nMAE (Mean Absolute Error):")
print(f"  - Entrenamiento: {mae_train_ridge:.4f} ($100k)")
print(f"  - Prueba: {mae_test_ridge:.4f} ($100k)")

# MAPE (Error Porcentual Absoluto Medio)
mape_train_ridge = mean_absolute_percentage_error(y_train, y_train_pred_ridge)
mape_test_ridge = mean_absolute_percentage_error(y_test, y_test_pred_ridge)
print(f"\nMAPE (Mean Absolute Percentage Error):")
print(f"  - Entrenamiento: {mape_train_ridge*100:.2f}%")
print(f"  - Prueba: {mape_test_ridge*100:.2f}%")

# Comparación con Regresión Lineal
print(f"\n{'='*50}")
print("COMPARACIÓN: RIDGE vs REGRESIÓN LINEAL")
print(f"{'='*50}")

mse_test_lr = mean_squared_error(y_test, y_test_pred_lr)
r2_test_lr = r2_score(y_test, y_test_pred_lr)
mae_test_lr = mean_absolute_error(y_test, y_test_pred_lr)

comparison_df = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'R²', 'MAE'],
    'Ridge': [mse_test_ridge, rmse_test_ridge, r2_test_ridge, mae_test_ridge],
    'Reg. Lineal': [mse_test_lr, np.sqrt(mse_test_lr), r2_test_lr, mae_test_lr]
})
print("\n", comparison_df.to_string(index=False))

# Validación cruzada
print(f"\n{'='*50}")
print("VALIDACIÓN CRUZADA (5-Fold)")
print(f"{'='*50}")

cv_scores = cross_val_score(ridge_model, X_train_scaled, y_train, 
                             cv=5, scoring='r2', n_jobs=-1)
print(f"\nR² Scores por fold: {cv_scores}")
print(f"R² promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 6. VISUALIZACIÓN DE RESULTADOS
print(f"\n{'='*70}")
print("6. VISUALIZACIÓN DE RESULTADOS")
print(f"{'='*70}")

# Importancia de características
feature_importance = pd.DataFrame({
    'Feature': california.feature_names,
    'Coefficient': ridge_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)


print("\nImportancia de características (coeficientes Ridge):")
print(feature_importance.to_string(index=False))

# 7. SISTEMA DE PREDICCIÓN FÁCIL
print(f"\n{'='*50}")
print("7. SISTEMA DE PREDICCIÓN PARA NUEVAS CASAS")
print(f"{'='*50}")

def predecir_precio_casa(MedInc, HouseAge, AveRooms, AveBedrms, 
                        Population, AveOccup, Latitude, Longitude):
    """
    Predice el precio de una casa dados sus características.
    
    Parámetros:
    - MedInc: Ingreso medio del área (en $10,000)
    - HouseAge: Edad media de las casas
    - AveRooms: Número promedio de habitaciones
    - AveBedrms: Número promedio de dormitorios
    - Population: Población del área
    - AveOccup: Ocupación promedio
    - Latitude: Latitud
    - Longitude: Longitud
    
    Retorna:
    - Precio predicho en dólares
    """
    # Crear array con las características
    nueva_casa = np.array([[MedInc, HouseAge, AveRooms, AveBedrms, 
                           Population, AveOccup, Latitude, Longitude]])
    
    # Normalizar usando el mismo scaler
    nueva_casa_scaled = scaler.transform(nueva_casa)
    
    # Predecir
    precio_predicho = ridge_model.predict(nueva_casa_scaled)[0]
    precio_en_dolares = precio_predicho * 100000
    
    return precio_predicho, precio_en_dolares

# Ejemplo de uso con valores promedio
print("\nEJEMPLO DE PREDICCIÓN:")
print("-" * 50)

# Usar valores promedio del dataset
ejemplo_medios = X.mean()
print("\nUsando valores promedio del dataset:")
for feature, valor in ejemplo_medios.items():
    print(f"  {feature}: {valor:.2f}")

precio_100k, precio_dolares = predecir_precio_casa(
    ejemplo_medios['MedInc'],
    ejemplo_medios['HouseAge'],
    ejemplo_medios['AveRooms'],
    ejemplo_medios['AveBedrms'],
    ejemplo_medios['Population'],
    ejemplo_medios['AveOccup'],
    ejemplo_medios['Latitude'],
    ejemplo_medios['Longitude']
)

print(f"\nPREDICCIÓN:")
print(f"  Precio: ${precio_100k:.2f} (en unidades de $100k)")
print(f"  Precio: ${precio_dolares:,.2f}")

# Ejemplo 2: Casa de lujo
print("\n" + "="*50)
print("EJEMPLO 2: Casa en área de alto ingreso")
print("="*50)

precio_100k_lujo, precio_dolares_lujo = predecir_precio_casa(
    MedInc=8.0,      # Alto ingreso
    HouseAge=20,     # Casa relativamente nueva
    AveRooms=7.0,    # Más habitaciones
    AveBedrms=1.2,   # Más dormitorios
    Population=1000,
    AveOccup=2.5,
    Latitude=37.8,   # Área de SF Bay
    Longitude=-122.4
)

print(f"\nCaracterísticas:")
print(f"  Ingreso medio: $80,000")
print(f"  Edad: 20 años")
print(f"  Habitaciones: 7.0")
print(f"  Población: 1,000")

print(f"\nPREDICCIÓN:")
print(f"  Precio: ${precio_100k_lujo:.2f} (en unidades de $100k)")
print(f"  Precio: ${precio_dolares_lujo:,.2f}")

# Ejemplo 3: Casa económica
print("\n" + "="*50)
print("EJEMPLO 3: Casa en área de menor ingreso")
print("="*50)

precio_100k_eco, precio_dolares_eco = predecir_precio_casa(
    MedInc=2.0,      # Menor ingreso
    HouseAge=40,     # Casa más antigua
    AveRooms=4.0,    # Menos habitaciones
    AveBedrms=1.0,
    Population=2000,
    AveOccup=3.5,
    Latitude=34.0,
    Longitude=-118.0
)

print(f"\nCaracterísticas:")
print(f"  Ingreso medio: $20,000")
print(f"  Edad: 40 años")
print(f"  Habitaciones: 4.0")
print(f"  Población: 2,000")

print(f"\nPREDICCIÓN:")
print(f"  Precio: ${precio_100k_eco:.2f} (en unidades de $100k)")
print(f"  Precio: ${precio_dolares_eco:,.2f}")

# 8. CONCLUSIONES
print(f"\n{'='*70}")
print("CONCLUSIONES Y RECOMENDACIONES")
print(f"{'='*70}")

conclusiones = f"""
RESUMEN DEL MODELO RIDGE:
-------------------------
✓ Modelo entrenado exitosamente con alpha óptimo = {best_alpha}
✓ R² Score en test: {r2_test_ridge:.4f} (explica {r2_test_ridge*100:.2f}% de la varianza)
✓ RMSE en test: ${rmse_test_ridge:.4f} (en $100k) = ${rmse_test_ridge*100000:,.2f}
✓ MAE en test: ${mae_test_ridge:.4f} (en $100k) = ${mae_test_ridge*100000:,.2f}

INTERPRETACIÓN:
--------------
- El modelo tiene un desempeño {'excelente' if r2_test_ridge > 0.8 else 'bueno' if r2_test_ridge > 0.6 else 'aceptable'}
- El error promedio de predicción es de aproximadamente ${mae_test_ridge*100000:,.2f}
- La regularización Ridge ayuda a prevenir overfitting
- Las características más importantes son: {', '.join(feature_importance['Feature'].head(3).tolist())}

VENTAJAS DE RIDGE vs REGRESIÓN LINEAL:
-------------------------------------
- Regularización L2 que penaliza coeficientes grandes
- Reduce el riesgo de overfitting
- Mejor generalización con nuevos datos
- Maneja mejor la multicolinealidad
"""

print(conclusiones)
print("="*70)

PREDICCIÓN DE PRECIOS DE CASAS EN CALIFORNIA - REGRESIÓN RIDGE

1. CARGANDO DATASET...

Dimensiones del dataset: (20640, 9)

Descripción del dataset:
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group popu...

Primeras filas:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.81